## Stability of caloric sufficiency w.r.t. diet changes
#### What we are trying to do is find what the % of meat consumption in diet needs to be for caloric sufficiency to remain the same from 2000 to 2050.
To achieve that we will write a simple function that calculates the ∆cal_suff that only depends on diet and then try to find its root(s).

In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import root

In [2]:
def f(x):
    '''example multivariate vectorial function'''
    return [np.log(x[0]**2) - x[1],
           x[1]**3 - 2 * x[0]]

root(f, x0=[-2, 1], method='hybr')

    fjac: array([[-0.91524006, -0.40290897],
       [ 0.40290897, -0.91524006]])
     fun: array([ 3.38906680e-12, -5.75206549e-12])
 message: 'The solution converged.'
    nfev: 17
     qtf: array([ 5.21627853e-10, -4.52168491e-09])
       r: array([ 3.96506486, -0.44605862, -3.36515374])
  status: 1
 success: True
       x: array([-0.58963483, -1.05650374])

In [691]:
all_countries = [
    
    'Argentina', 'Australia', 'Bulgaria',
    'Belize', 'Brazil', 'Canada', 'Denmark',
    'France', 'Hungary', 'Kazakhstan',
    'Lithuania', 'Latvia', 'Paraguay',
    'Ukraine', 'Uruguay', 'United States of America',
    'Belarus', 'Botswana', 'Estonia',
    'Finland', 'Georgia', 'Iran', 'Libya',
    'Lesotho', 'Moldova', 'Mongolia',
    'Norway', 'New Zealand', 'Poland',
    'Portugal', 'Russia', 'Swaziland',
    'Tunisia',
    'United Arab Emirates', 'Belgium',
    'Brunei', 'Cyprus', 'Djibouti', 'Algeria',
    'Gambia', 'Iraq', 'Israel', 'Jordan',
    'Japan', 'South Korea', 'Kuwait',
    'Lebanon', 'Montenegro',
    'Mauritania', 'Netherlands', 'Saudi Arabia', 'Singapore', 'Trinidad and Tobago', 'Yemen',
    'Afghanistan', 'Angola', 'Burundi',
    'Bangladesh', 'Cameroon',
    'Democratic Republic of the Congo',
    'Republic of Congo', 'Dominican Republic', 'Egypt', 'Eritrea', 'Ethiopia',
    'Ghana', 'Guinea Bissau',
    'Guatemala', 'Haiti', 'India', 'Jamaica',
    'Kenya', 'Sri Lanka', 'Madagascar',
    'Mozambique', 'Malawi', 'Niger',
    'Nigeria', 'Nepal', 'Oman', 'Pakistan',
    'Philippines', 'Puerto Rico', 'North Korea', 'Rwanda', 'El Salvador',
    'Syria', 'Togo', 'Uganda',
    'Benin', 'Burkina Faso', 'Bolivia',
    'China', 'Ivory Coast', 'Colombia',
    'Costa Rica', 'United Kingdom',
    'Guinea', 'Honduras', 'Indonesia',
    'Ireland', 'Luxembourg', 'Mexico',
    'Malaysia', 'Panama', 'Sudan',
    'Senegal', 'Sierra Leone', 'Tajikistan',
    'Tanzania', 'Uzbekistan', 'Vietnam',
    'Zambia', 'Gabon', 'Equatorial Guinea',
    'Morocco', 'Peru',
    'Albania', 'Austria', 'Azerbaijan',
    'Bosnia and Herzegovina', 'Chile',
    'Czech Republic', 'Germany', 'Spain',
    'Greece', 'Croatia', 'Italy',
    'Kyrgyzstan', 'Laos', 'Macedonia',
    'Mali', 'Nicaragua', 'Papua New Guinea', 'Romania', 'Somalia',
    'Serbia', 'Slovakia', 'Slovenia',
    'Chad', 'Thailand', 'Turkmenistan',
    'East Timor', 'Turkey', 'Venezuela',
    'Armenia', 'Bhutan', 'Central African Republic', 'Switzerland', 'Cuba',
    'Ecuador', 'Guyana', 'Cambodia',
    'Liberia', 'Myanmar', 'Namibia',
    'Suriname', 'Sweden', 'South Africa',
    'Zimbabwe'
]

countries_names = pd.read_csv('country_names.csv',encoding='latin-1').rename({'Country Code': 'ISO3'}, axis=1)
countries_names = countries_names.iloc[[e in all_countries for e in list(countries_names['name'].values)], :]

diet_2000 = pd.read_csv('Consumption_2000_FAOSTAT.csv')
LS_2000_df = diet_2000[diet_2000['Item']=='Animal Products'][['Country','Value']].merge(countries_names[['name','ISO3']].drop_duplicates(), right_on='name',left_on='Country',how='left')
# kcal/capita/day to cal/capita
LS_2000_df['LS2000_percapita'] = LS_2000_df['Value'].apply(lambda x:x*1e3*365.25)
LS_2000_df.head()

,Country,Value,name,ISO3,LS2000_percapita
0,Afghanistan,275,Afghanistan,AFG,100443750.0
1,Albania,674,Albania,ALB,246178500.0
2,Algeria,277,Algeria,DZA,101174250.0
3,Angola,153,Angola,AGO,55883250.0
4,Antigua and Barbuda,701,NaN,NaN,256040250.0


In [458]:
# function that returns the population from a dataframe by year
def get_population(X, year=2050):
    return int(X['population_'+str(year)].sum())

# function that returns the total production from a dataframe by year
def get_production(X, year=2050):
    return X['calories_'+str(year)].sum()

# function that calculates the caloric sufficiency. it is fed by a dataframe [to preserve generality]
def cal_suff_small(X, cntry=None, year=2050, food_waste=.19, feed_food_factor=2.3, perc_feed = .29, diet=.19, pop_fact=1., ADER=2320*1e3*365.25):
    
    
    conso = 1-.19
    perc_food = conso-perc_feed
    
    prod = get_production(X, year=year)
    food = perc_food * prod
    feed_now_food = 0.
    if cntry in LS_2000_df.ISO3.unique():
        LS = LS_2000_df[LS_2000_df.ISO3==cntry]['LS2000_percapita'].values * get_population(X, year=2000)
        
    else:
        LS = (1.1483e15*1e3)  * get_population(X, year=2000) / 5976296907
    if year==2050:
        LS_2000 = LS
        LS = diet * (food + 2.3 * LS_2000) / (1 + diet * (feed_food_factor - 1))
        feed_now_food = 2.3 * (LS_2000) - feed_food_factor * (LS)
    
    production = (1-food_waste) / (1-.19) * (food + LS + feed_now_food)
    demand = get_population(X, year=year) * pop_fact * ADER
    
    return production/demand

In [672]:
ssp = 2
diets = [0.23728686354880824, 0.22905550297316568, 0.20972663721021714, 0.20811659841733302, 0.21198181981961245]
data_path = 'outputs/compare/'
df = pd.read_csv(data_path+'ssp'+str(ssp)+'_compare_new.csv')

my_diet = diets[ssp-1]
cal_suff_small(df[df.ISO3=='USA'], cntry='USA', year=2050, diet=my_diet)

array([3.85906246])

### Keeping caloric sufficiency constant through diet

In [671]:
def delta_cal_suff(x, country=None):
#     x[0] = perc_diet : percentage of livestock in diet

    df_dup = df.copy()
    if country:
        df_dup = df[df['ISO3']==country].copy()
    
    cal_suff_2000 = cal_suff_small(df_dup, cntry=country, year=2000)
    cal_suff_2050 = cal_suff_small(df_dup, cntry=country, year=2050, diet=x[0])
        
    return [cal_suff_2050 - cal_suff_2000]


delta_cal_suff(x=[.19])

[-0.519914259902474]

In [667]:
# solution to the problem using MINIPACK's hybrd solver

sol = root(delta_cal_suff, x0=[.19], method='hybr').x
sol, delta_cal_suff(sol), my_diet

(array([-0.06485151]), [1.9984014443252818e-15], 0.22905550297316568)

In [670]:
# assetion of the solution's validity

cal_suff_small(df, diet=.101),cal_suff_small(df, diet=my_diet)

(1.5839436884285485, 1.3807627859654794)

### Keeping caloric sufficiency constant through yield improvement

In [640]:
def delta_cal_suff(x, country=None):
#     x = yield boost coefficient

    df_dup = df.copy()
    if country:
        df_dup = df[df['ISO3']==country].copy()
    
    df_dup['cal_per_ha_pred'] = df_dup['cal_per_ha_pred'].apply(lambda y: x[0] * y)
    df_dup.loc[:, 'calories_2050'] = df_dup['%cropland_2050'] * df_dup['cal_per_ha_pred'] * df_dup['ha_per_pixel']
    
    cal_suff_2000 = cal_suff_small(df_dup, cntry=country, year=2000)
    cal_suff_2050 = cal_suff_small(df_dup, cntry=country, year=2050, diet=my_diet)
        
    return [cal_suff_2050 - cal_suff_2000]


delta_cal_suff(x=[1.])

[-0.5761326244736567]

In [641]:
# solution to the problem using MINIPACK's hybrd solver

sol = root(delta_cal_suff, x0=[1.0], method='hybr').x
sol, delta_cal_suff(sol)

(array([1.52940783]), [4.440892098500626e-16])

In [642]:
# assetion of the solution's validity
df_test = df.copy()
df_test['cal_per_ha_pred'] = df_test['cal_per_ha_pred'].apply(lambda y: sol[0] * y)
df_test.loc[:, 'calories_2050'] = df_test['%cropland_2050'] * df_test['cal_per_ha_pred'] * df_test['ha_per_pixel']

cal_suff_small(df_test, diet=my_diet), cal_suff_small(df_test, year=2000, diet=.19)

(1.9568954104391358, 1.9568954104391354)

In [643]:
(df_test['cal_per_ha_pred']).mean()/1e+9

10.454255964586562

### Keeping caloric sufficiency constant through cropland expansion

In [627]:
def expansion(x, y):
    if x==0:
        return 0
    else:
        return max(min(1., x+(y-1)),0)
    
def delta_cal_suff(x, country=None):
#     x = cropland expansion coefficienct

    df_dup = df.copy()
    if country:
        df_dup = df[df['ISO3']==country].copy()
    
    df_dup['%cropland_2050'] = df_dup['%cropland_2050'].fillna(0)
        
    df_dup['%cropland_2050'] = df_dup['%cropland_2050'].apply(lambda y: expansion(y, x[0]))
    df_dup['cal_per_ha_pred'] = df_dup['cal_per_ha_2000']
    df_dup.loc[:, 'calories_2050'] = df_dup['%cropland_2050'] * df_dup['cal_per_ha_pred'] * df_dup['ha_per_pixel']
    
    cal_suff_2000 = cal_suff_small(df_dup, cntry=country, year=2000)
    cal_suff_2050 = cal_suff_small(df_dup, cntry=country, year=2050, diet=my_diet)
        
    return [cal_suff_2050 - cal_suff_2000]


delta_cal_suff(x=[1.])

[-0.6756365835425306]

In [628]:
# solution to the problem using MINIPACK's hybrd solver

sol = root(delta_cal_suff, x0=[1.], method='hybr').x
sol, delta_cal_suff(sol)

(array([1.38428966]), [6.661338147750939e-16])

In [637]:
# assetion of the solution's validity
df_test = df.copy()
df_test['%cropland_2050'] = df_test['%cropland_2050'].fillna(0)
df_test['%cropland_2050'] = df_test['%cropland_2050'].apply(lambda y: expansion(y, sol[0]))
df_test['cal_per_ha_pred'] = df_test['cal_per_ha_2000']
df_test.loc[:, 'calories_2050'] = df_test['%cropland_2050'] * df_test['cal_per_ha_pred'] * df_test['ha_per_pixel']

cal_suff_small(df_test, diet=my_diet), cal_suff_small(df_test, year=2000, diet=.19)

(1.956895410439136, 1.9568954104391354)

In [636]:
(df_test['%cropland_2050'] * df_test['ha_per_pixel']).sum()/1e+9, (df['%cropland_2050'] * df['ha_per_pixel']).sum()/1e+9

(5.448334964532038, 2.6597913135669318)

### Keeping caloric sufficiency constant through population change

In [673]:
def delta_cal_suff(x, country=None):
#     x = population distortion factor

    df_dup = df.copy()
    if country:
        df_dup = df[df['ISO3']==country].copy()
        
    cal_suff_2000 = cal_suff_small(df_dup, cntry=country, year=2000)
    cal_suff_2050 = cal_suff_small(df_dup, cntry=country, year=2050, diet=my_diet, pop_fact=x[0])
        
    return [cal_suff_2050 - cal_suff_2000]


delta_cal_suff(x=[1.])

[-0.576132624473656]

In [674]:
# solution to the problem using MINIPACK's hybrd solver

sol = root(delta_cal_suff, x0=[1.0], method='hybr').x
sol, delta_cal_suff(sol)

(array([0.70558844]), [-2.220446049250313e-16])

In [681]:
# assetion of the solution's validity

cal_suff_small(df, diet=my_diet, pop_fact=sol[0]), cal_suff_small(df, year=2000, diet=.19)

(1.5780146125319765, 1.3807627859654794)

In [682]:
# population change from 2050 projection and solution
from decimal import Decimal

'%.2E' % Decimal((1-sol[0]) * get_population(df))

'2.42E+09'

### Keeping caloric sufficiency constant through food waste, diet and yield improvement

In [662]:
# we make the assumption that food waste is reduced by half and infer the remainder of the variable values

def delta_cal_suff(x, country=None):
#     x[0] = perc_feed : percentage of production that goes to feeding LS
#     x[1] = yield boost coefficient
    waste = .19 * .4
    df_dup = df.copy()
    if country:
        df_dup = df[df['ISO3']==country].copy()
    
    df_dup['cal_per_ha_pred'] = df_dup['cal_per_ha_pred'].apply(lambda y: x[0] * y)
    df_dup.loc[:, 'calories_2050'] = df_dup['%cropland_2050'] * df_dup['cal_per_ha_pred'] * df_dup['ha_per_pixel']
    
    cal_suff_2000 = cal_suff_small(df_dup, cntry=country, year=2000)
    cal_suff_2050 = cal_suff_small(df_dup, cntry=country, year=2050, \
                                   feed_food_factor=1.75, diet=my_diet*.75, food_waste=waste)
    
    return [cal_suff_2050 - cal_suff_2000]


delta_cal_suff(x=[1.])

[-0.14609470188231466]

In [663]:
# solution to the problem using MINIPACK's hybrd solver

sol = root(delta_cal_suff, x0=[1.], method='hybr').x
sol, delta_cal_suff(sol)

(array([1.10236483]), [6.661338147750939e-16])

In [664]:
# assetion of the solution's validity
df_test = df.copy()
df_test['cal_per_ha_pred'] = df_test['cal_per_ha_pred'].apply(lambda y: sol[0] * y)
df_test.loc[:, 'calories_2050'] = df_test['%cropland_2050'] * df_test['cal_per_ha_pred'] * df_test['ha_per_pixel']

cal_suff_small(df_test, diet=my_diet*.5, food_waste=.19*.9, feed_food_factor=2.0), cal_suff_small(df_test, year=2000, diet=.19)

(1.7782511926309887, 1.9568954104391354)

In [665]:
(df_test['cal_per_ha_pred']).mean()/1e+9

7.53520667748413

### Keeping caloric sufficiency constant using feed to food factor and food waste


In [689]:
def delta_cal_suff(x, country=None):
#     x = food_waste

    df_dup = df.copy()
    if country:
        df_dup = df[df['ISO3']==country].copy()
        
    cal_suff_2000 = cal_suff_small(df_dup, cntry=country, year=2000)
    cal_suff_2050 = cal_suff_small(df_dup, cntry=country, year=2050, diet=my_diet*.33, feed_food_factor=1.8, \
                                   food_waste=x[0])
        
    return [cal_suff_2050 - cal_suff_2000]


delta_cal_suff(x=[.19])

[-0.2671593578539513]

In [690]:
# solution to the problem using MINIPACK's hybrd solver

sol = root(delta_cal_suff, x0=[.19], method='hybr').x
sol, delta_cal_suff(sol)

(array([0.06193321]), [-2.220446049250313e-16])

In [465]:
# assetion of the solution's validity

cal_suff_small(df, diet=my_diet, food_waste=sol[0], feed_food_factor=1.15), cal_suff_small(df, year=2000, diet=.19)

(1.9568954104391352, 1.9568954104391354)